In [ ]:
import sys
import json
import os
from datetime import datetime
import pyarrow.parquet as pq
import time
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sankey import Sankey
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

# Import tableone
try:
    from tableone import TableOne, load_dataset
except (ModuleNotFoundError, ImportError):
    # install on Colab
    !pip install tableone
    from tableone import TableOne, load_dataset

In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import config
config = config.load_config()

## please change site_name, tables_path, output_path, and file_type 

In [ ]:
# Access configuration parameters
site_name = config['site_name']
tables_path = config['tables_path']
file_type = config['file_type']
output_path = os.path.join("..", "output", "final")

# Make sure the directory exists; if not, create it
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Print the configuration parameters
print(f"Site Name: {site_name}")
print(f"Tables Path: {tables_path}")
print(f"Output path: {output_path}")
print(f"File Type: {file_type}")

In [ ]:
## Confirm that these are the correct paths
adt_filepath = f"{tables_path}/clif_adt.{file_type}"
hospitalization_filepath = f"{tables_path}/clif_hospitalization.{file_type}"
patient_filepath = f"{tables_path}/clif_patient.{file_type}"

In [ ]:
def read_data(filepath, filetype):
    start_time = time.time()  # Record the start time
    file_name = os.path.basename(filepath) 
    if filetype == 'csv':
        df = pd.read_csv(filepath)
    elif filetype == 'parquet':
        table = pq.read_table(filepath)
        df = table.to_pandas()
    else:
        raise ValueError("Unsupported file type. Please provide either 'csv' or 'parquet'.")
    
    end_time = time.time()  # Record the end time
    load_time = end_time - start_time  # Calculate the loading time
    
    # Calculate the size of the loaded dataset in MB
    dataset_size_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)
    print(f"File name: {file_name}")
    print(f"Time taken to load the dataset: {load_time:.2f} seconds")
    print(f"Size of the loaded dataset: {dataset_size_mb:.2f} MB\n")
    
    return df

adt = read_data(adt_filepath, file_type)
hosp = read_data(hospitalization_filepath, file_type)
patient = read_data(patient_filepath,file_type)

In [ ]:
print("number of unique hospitalization_id:",hosp["hospitalization_id"].nunique())

In [ ]:
missing_admission_dttm = hosp['admission_dttm'].isnull().sum()
print("Missing admission_dttm:", missing_admission_dttm)

In [ ]:
hosp2 = hosp[["patient_id","hospitalization_id","admission_dttm","discharge_dttm","age_at_admission"]].copy()
hosp2['admission_dttm'] = pd.to_datetime(hosp2['admission_dttm'])
hosp2['discharge_dttm'] = pd.to_datetime(hosp2['discharge_dttm'])
hosp2 = hosp2[(hosp2['admission_dttm'].dt.year >= 2020) & 
                   (hosp2['admission_dttm'].dt.year <= 2021) & 
                   (hosp2['age_at_admission'] >=18)&
                    (hosp2['age_at_admission'] <=119)&
                     (hosp2['hospitalization_id'].isin(adt[adt['location_category'].str.lower() == "icu"]['hospitalization_id'].unique()))]

df = pd.merge(hosp2[["patient_id","hospitalization_id"]],
                  adt[["hospitalization_id","in_dttm","out_dttm","location_category"]],
             on="hospitalization_id",how="left")

print("number of unique hospitalization_id:",df["hospitalization_id"].nunique())
print("number of rows in df:",len(df))
df.head()

In [ ]:
df["location_category"] = df["location_category"].str.lower().str.strip()
# Mapping dictionary to update the location_category
location_mapping = {
    'rehab': 'Other',
    'l&d': 'Ward',
    'psych': 'Other',
    'radiology': 'Other',
    'dialysis': 'Other',
    'hospice': 'Other',
    'stepdown':'ICU',
    "icu": "ICU",
    "er": "ER",
    "ward":"Ward",
    "procedural":"Procedural",
    "other":"Other",
    'imaging':'Other',
    'ltach':'Other'
}

# Apply the mapping to the 'location_category' column
df['location_category'] = df['location_category'].replace(location_mapping)
df["location_category"].value_counts()

In [ ]:
# Step 1: Sort by hospitalization_id and in_dttm
df = df.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])
df = df[df['location_category']!= "Other"] #removing "Other from the study"
# Apply the function to each group of hospitalization_id
df = df.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])
#stitch together ICUs if ICU --> procedure --> ICU
df['shifted_location'] = df['location_category'].shift(1)
df['new_location'] = (df['location_category'] != df['shifted_location']).cumsum()
df = df.groupby(['hospitalization_id','new_location']).agg(
    in_dttm=pd.NamedAgg(column='in_dttm', aggfunc='min'),
    out_dttm=pd.NamedAgg(column='out_dttm', aggfunc='max'),
    location_category=pd.NamedAgg(column='location_category', aggfunc='first'), 
    patient_id=pd.NamedAgg(column='patient_id', aggfunc='first')
).reset_index()

# Assuming 'df' is your DataFrame and it's sorted appropriately
df.sort_values(['hospitalization_id', 'in_dttm'], inplace=True)  # Replace 'timestamp_column' with the actual column name that orders your events chronologically.
# Use groupby and shift to get previous and next 'location_category' within each hospitalization_id
df['prev_category'] = df.groupby('hospitalization_id')['location_category'].shift(1)
df['next_category'] = df.groupby('hospitalization_id')['location_category'].shift(-1)
# Create a boolean mask for 'Procedural' rows sandwiched between 'ICU' rows
mask = (
    (df['location_category'] == 'Procedural') &
    (df['prev_category'] == 'ICU') &
    (df['next_category'] == 'ICU')
)
# Remove the rows where the mask is True
df = df[~mask].reset_index(drop=True)
# Drop the helper columns
df.drop(columns=['prev_category', 'next_category'], inplace=True)

print("Number of unique hospitalization_id:", df["hospitalization_id"].nunique())
print("Number of rows in df:",len(df))

In [ ]:
def remove_short_stays(df):
    # Ensure the in_dttm and out_dttm columns are in datetime format
    df['in_dttm'] = pd.to_datetime(df['in_dttm'], errors='coerce')
    df['out_dttm'] = pd.to_datetime(df['out_dttm'], errors='coerce')

    # Directly filter rows based on the time difference without creating a new column
    mask = (df['out_dttm'] - df['in_dttm']) >= pd.Timedelta(hours=1)
    df_filtered = df[mask]
    
    return df_filtered

# Sort the DataFrame once
df = df.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])

# Remove short stays
df = remove_short_stays(df)

# Efficiently shift and calculate new locations
df['shifted_location'] = df['location_category'].shift(1)
df['new_location'] = (df['location_category'] != df['shifted_location']).cumsum()

# Group by hospitalization_id and new_location in one operation, minimize memory use
df = df.groupby(['hospitalization_id', 'new_location'], as_index=False).agg(
    in_dttm=('in_dttm', 'min'),
    out_dttm=('out_dttm', 'max'),
    location_category=('location_category', 'first'),
    patient_id=('patient_id', 'first')
)

# Print summary
print("Number of unique hospitalization_id:", df["hospitalization_id"].nunique())
print("Number of rows in df:", len(df))


In [ ]:
import pandas as pd

# Convert in_dttm and out_dttm columns to datetime format
df['in_dttm'] = pd.to_datetime(df['in_dttm'])
df['out_dttm'] = pd.to_datetime(df['out_dttm'])

# Sort the DataFrame
df = df.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])

# Function to mark rows as procedural if 'Procedural' is immediately before the second ICU
def mark_procedural_before_second_icu(df):
    # Find ICU rows
    df['is_icu'] = df['location_category'] == 'ICU'
    
    # Create a group-level cumulative sum to count ICU admissions
    df['icu_count'] = df.groupby(['patient_id', 'hospitalization_id'])['is_icu'].cumsum()

    # Create a 'procedural' column that is initially False
    df['procedural'] = False

    # Identify rows where the second ICU occurs
    for hosp_id, group in df.groupby(['patient_id', 'hospitalization_id']):
        # Find the indices of ICU rows
        icu_rows = group[group['location_category'] == 'ICU'].index

        # If there are at least two ICU admissions
        if len(icu_rows) > 1:
            second_icu_index = icu_rows[1]  # Second ICU occurrence
            
            # Check the row immediately before the second ICU
            if second_icu_index - 1 in group.index and group.loc[second_icu_index - 1, 'location_category'] == 'Procedural':
                # Mark the row as procedural
                df.loc[second_icu_index - 1, 'procedural'] = True
                
    # Propagate procedural == True to all rows in the same hospitalization_id if any row is marked as True
    df['procedural'] = df.groupby('hospitalization_id')['procedural'].transform('max')

    # Drop helper columns
    df.drop(columns=['is_icu', 'icu_count'], inplace=True)

    return df

# Further transformations as needed
df2 = df.copy()

# Ensure dataset is aggregated correctly
df2['shifted_location'] = df2['location_category'].shift(1)
df2['new_location'] = (df2['location_category'] != df2['shifted_location']).cumsum()

df2 = df2.groupby(['hospitalization_id', 'new_location']).agg(
    in_dttm=('in_dttm', 'min'),
    out_dttm=('out_dttm', 'max'),
    location_category=('location_category', 'first'),
    patient_id=('patient_id', 'first')
).reset_index()

# Apply the function to mark procedural rows
df2 = mark_procedural_before_second_icu(df2)

# Output the updated DataFrame
print("Number of unique hospitalization_id:", df2["hospitalization_id"].nunique())
print("Number of rows in df:", len(df2))
print("Number of hospitalizations marked as procedural:", df2[df2["procedural"] == True]["hospitalization_id"].nunique())
df2[['patient_id', 'hospitalization_id', 'location_category', 'procedural']].head()

In [ ]:
# Convert the 'in_dttm' and 'out_dttm' columns to datetime format (vectorized)
df2['in_dttm'] = pd.to_datetime(df2['in_dttm'])
df2['out_dttm'] = pd.to_datetime(df2['out_dttm'])

# Sort the DataFrame by hospitalization_id and datetime columns
df2 = df2.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])

# Filter directly for ICU visits
icu_visits = df2[df2['location_category'] == "ICU"]

# Get the unique hospitalization_ids that had an ICU visit
icu_visit_ids = icu_visits['hospitalization_id'].unique()

# Filter the entire DataFrame to include only rows with those hospitalization_ids
icu_df = df2[df2['hospitalization_id'].isin(icu_visit_ids)]

# Sort the filtered DataFrame again if needed
icu_df = icu_df.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])

# Output the resulting DataFrame and summary statistics
print("Number of unique hospitalization_id:", icu_df["hospitalization_id"].nunique())
print("Number of rows in df:", len(icu_df))


In [ ]:
def drop_deaths_during_icu(df, patient_df):
    # Merge death_dttm from patient data into icu_df, only keep relevant columns in patient_df
    df = df.merge(patient_df[['patient_id', 'death_dttm']], on='patient_id', how='left')

    # Convert 'death_dttm', 'in_dttm', and 'out_dttm' to datetime format (more efficient bulk conversion)
    df[['death_dttm', 'in_dttm', 'out_dttm']] = df[['death_dttm', 'in_dttm', 'out_dttm']].apply(pd.to_datetime, errors='coerce')

    # Filter directly for the first ICU admission per hospitalization_id
    icu_first_admission = df[df['location_category'] == 'ICU'].sort_values('in_dttm').drop_duplicates('hospitalization_id')

    # Identify hospitalization_id where the patient died during ICU admission
    death_during_icu = icu_first_admission.loc[
        (icu_first_admission['death_dttm'].notnull()) &
        (icu_first_admission['death_dttm'] >= icu_first_admission['in_dttm']) &
        (icu_first_admission['death_dttm'] <= icu_first_admission['out_dttm']),
        'hospitalization_id'
    ]

    # Filter out rows where the patient died during ICU admission
    df_cleaned = df[~df['hospitalization_id'].isin(death_during_icu)]
    dropped_df = df[df['hospitalization_id'].isin(death_during_icu)]

    return df_cleaned, dropped_df

# Sorting the icu_df by 'hospitalization_id' and 'in_dttm'
icu_df = icu_df.sort_values(by=['hospitalization_id', 'in_dttm'])

# Apply the function
icu_df2, dropped_deaths_df = drop_deaths_during_icu(icu_df, patient)

# Output statistics
print("Number of unique hospitalization_id with death during ICU:", dropped_deaths_df["hospitalization_id"].nunique())
print("Number of unique hospitalization_id after removing death during ICU:", icu_df2["hospitalization_id"].nunique())
print("Number of rows in df:", len(icu_df2))


In [ ]:
def drop_direct_icu_discharges(df):
    # Sort the DataFrame by hospitalization_id and in_dttm if not already sorted
    df = df.sort_values(by=['hospitalization_id', 'in_dttm'])

    # Identify the first ICU admission for each hospitalization_id
    first_icu = df[df['location_category'] == 'ICU'].groupby('hospitalization_id').head(1)

    # Identify the last row for each hospitalization_id
    last_row = df.groupby('hospitalization_id').tail(1)

    # Correct the logic: Check where the first ICU row is also the last row in the hospitalization
    visits_to_drop = first_icu[first_icu.index.isin(last_row.index)]['hospitalization_id']

    # Filter out the rows for hospitalizations where patients were discharged directly from ICU
    df_cleaned = df[~df['hospitalization_id'].isin(visits_to_drop)]
    dropped_df = df[df['hospitalization_id'].isin(visits_to_drop)]

    # Drop helper columns (if any)
    df_cleaned.drop(columns=['is_icu'], inplace=True, errors='ignore')

    return df_cleaned, dropped_df

# Sort the input DataFrame
icu_df2 = icu_df2.sort_values(by=['hospitalization_id', 'in_dttm', 'out_dttm'])

# Apply the optimized function
icu_df3, dropped_df_icu_discharge = drop_direct_icu_discharges(icu_df2)

# Output the counts and dropped DataFrame
print("Number of unique hospitalization_id after filtering:", icu_df3["hospitalization_id"].nunique())
print("Number of rows in df:", len(icu_df3))
print("Number of unique patient_id dropped:", dropped_df_icu_discharge["patient_id"].nunique())
print("Number of unique hospitalization_id dropped:", dropped_df_icu_discharge["hospitalization_id"].nunique())


In [ ]:
#redundant but making sure that the location_category is grouped together 
icu_df4 = icu_df3.sort_values(by=['hospitalization_id', 'in_dttm','out_dttm'])
icu_df4['shifted_location'] = icu_df4['location_category'].shift(1)
icu_df4['new_location'] = (icu_df4['location_category'] != icu_df4['shifted_location']).cumsum()

icu_df4 = icu_df4.groupby(['hospitalization_id','new_location']).agg(
    earliest_location_start=pd.NamedAgg(column='in_dttm', aggfunc='min'),
    latest_location_end=pd.NamedAgg(column='out_dttm', aggfunc='max'),
    location_category=pd.NamedAgg(column='location_category', aggfunc='first'), 
    patient_id=pd.NamedAgg(column='patient_id', aggfunc='first'),
    procedural = pd.NamedAgg(column='procedural', aggfunc='first')
).reset_index()

print("Number of unique hospitalization_id:", icu_df4["hospitalization_id"].nunique())
icu_df4.head()

In [ ]:
def drop_intermediate_er(df):
    # Sort the DataFrame to ensure it's ordered correctly by hospitalization_id and location timing
    df = df.sort_values(by=['hospitalization_id', 'earliest_location_start', 'latest_location_end'])

    # Create a mask for rows that are the first row within each hospitalization_id group
    first_row_mask = df.groupby('hospitalization_id').cumcount() == 0

    # Create a mask for rows where the location_category is 'ER'
    er_mask = df['location_category'] == 'ER'

    # Find the hospitalization_ids where 'ER' occurs after the first row
    er_after_first_row = df[~first_row_mask & er_mask]['hospitalization_id'].unique()

    # Filter the DataFrame to drop those hospitalization_ids
    df_cleaned = df[~df['hospitalization_id'].isin(er_after_first_row)]
    dropped_df = df[df['hospitalization_id'].isin(er_after_first_row)]

    return df_cleaned, dropped_df

# Sort and apply the drop function
icu_df4 = icu_df4.sort_values(by=['hospitalization_id', 'earliest_location_start', 'latest_location_end'])
icu_df5, dropped_df = drop_intermediate_er(icu_df4)

# Convert to datetime and calculate location hours
icu_df5['earliest_location_start'] = pd.to_datetime(icu_df5['earliest_location_start'])
icu_df5['latest_location_end'] = pd.to_datetime(icu_df5['latest_location_end'])
icu_df5['location_hours'] = (icu_df5['latest_location_end'] - icu_df5['earliest_location_start']).dt.total_seconds() / 3600

# Output the results
print("Number of rows in df:", len(icu_df5))
print("Number of unique hospitalization_id:", icu_df5["hospitalization_id"].nunique())
icu_df5.head()


# ICU readmission

In [ ]:
def calculate_icu_readmission_time(df):
    # Initialize a new column 'ICU_readmission_hour' with None
    df['ICU_readmission_hour'] = None

    # Function to process each group
    def process_group(group):
        # Find the rows where location_category is ICU
        icu_rows = group[group['location_category'] == 'ICU']

        # Check if there are at least two ICU events
        if len(icu_rows) > 1:
            # Sort by earliest_start2 to ensure correct order
            icu_rows = icu_rows.sort_values(by='earliest_location_start')

            # Get the first and second ICU event
            first_icu = icu_rows.iloc[0]
            second_icu = icu_rows.iloc[1]

            # Calculate the time difference between first and second ICU events in hours
            time_diff = (second_icu['earliest_location_start'] - first_icu['latest_location_end']).total_seconds() / 3600

            # Update the entire group with the calculated time difference
            df.loc[group.index, 'ICU_readmission_hour'] = time_diff

    # Apply the function to each group of 'person_id' and 'custom_visit_occurrence2'
    df.groupby(['patient_id', 'hospitalization_id']).apply(process_group)

    return df

icu_df6 = calculate_icu_readmission_time(icu_df5)
print("number of rows in df:",len(icu_df6))
print("Number of unique hospitalization_id:", icu_df6["hospitalization_id"].nunique())

In [ ]:
# Identify hospitalization_ids where ICU_readmission_hour is less than 0
#This is in place incase there are errors in the adt, should be a very small number if any 
hosp_ids_to_drop = icu_df6[icu_df6["ICU_readmission_hour"] <=0]["hospitalization_id"].unique()

# Drop rows with those hospitalization_ids
icu_df7 = icu_df6[~icu_df6["hospitalization_id"].isin(hosp_ids_to_drop)]

# Check the result
print("Number of unique hospitalization_id after filtering:", icu_df7["hospitalization_id"].nunique())
print("Number of rows in df after filtering:", len(icu_df7))

In [ ]:
#pd.set_option('display.max_rows',None)

In [ ]:
#icu_df6[icu_df6["ICU_readmission_hour"]>0][:100]

In [ ]:
icu_df7[icu_df7["procedural"]==True]

In [ ]:
# Select relevant columns
icu_final = icu_df7[["patient_id", "hospitalization_id", "ICU_readmission_hour","procedural"]]

# Drop duplicates based on 'custom_visit_occurrence2'
icu_final = icu_final.drop_duplicates(subset=["hospitalization_id"])

# Display the first few rows of the result
print("Number of rows in df:",len(icu_final))
print("Number of unique patient_id:", icu_final["patient_id"].nunique())
print("Number of unique hospitalization_id:", icu_final["hospitalization_id"].nunique())
icu_final.head()

# Create ICU Readmission Table

In [ ]:
total_unique_visits = icu_final["hospitalization_id"].nunique()
unique_visits_with_procedures = icu_final[icu_final["procedural"]==True]["hospitalization_id"].nunique()
unique_visits_without_procedures = icu_final[icu_final["procedural"]==False]["hospitalization_id"].nunique()
all_icu_readmission = icu_final[(icu_final["ICU_readmission_hour"]>0)]["hospitalization_id"].nunique()
all_icu_readmission_6hours = icu_final[(icu_final["ICU_readmission_hour"]<6)]["hospitalization_id"].nunique()
all_icu_readmission_24hours = icu_final[(icu_final["ICU_readmission_hour"]<24)]["hospitalization_id"].nunique()
all_icu_readmission_48hours = icu_final[(icu_final["ICU_readmission_hour"]<48)]["hospitalization_id"].nunique()
all_icu_readmission_72hours = icu_final[(icu_final["ICU_readmission_hour"]<72)]["hospitalization_id"].nunique()
unplanned_icu_readmission = icu_final[(icu_final["procedural"]==False)&(icu_final["ICU_readmission_hour"]>0)]["hospitalization_id"].nunique()
unplanned_icu_readmission_6hours = icu_final[(icu_final["procedural"]==False)&(icu_final["ICU_readmission_hour"]<6)]["hospitalization_id"].nunique()
unplanned_icu_readmission_24hours = icu_final[(icu_final["procedural"]==False)&(icu_final["ICU_readmission_hour"]<24)]["hospitalization_id"].nunique()
unplanned_icu_readmission_48hours = icu_final[(icu_final["procedural"]==False)&(icu_final["ICU_readmission_hour"]<48)]["hospitalization_id"].nunique()
unplanned_icu_readmission_72hours = icu_final[(icu_final["procedural"]==False)&(icu_final["ICU_readmission_hour"]<72)]["hospitalization_id"].nunique()


print("Number of unique hospitalization_id:", total_unique_visits)
print("Planned readmission: Number of unique hospitalization_id with procedures between ICU readmission:", unique_visits_with_procedures)
print("Unplanned readmission: Number of unique hospitalization_id without any procedures between ICU readmission:", unique_visits_without_procedures)

all_icu_read = round((all_icu_readmission / total_unique_visits)*100,2)
all_6hrs = round((all_icu_readmission_6hours/total_unique_visits)*100,2)
all_24hrs = round((all_icu_readmission_24hours/total_unique_visits)*100,2)
all_48hrs = round((all_icu_readmission_48hours/total_unique_visits)*100,2)
all_72hrs = round((all_icu_readmission_72hours/total_unique_visits)*100,2)

unplanned_icu_read = round((unplanned_icu_readmission / unique_visits_without_procedures)*100,2)
unplanned_6hrs = round((unplanned_icu_readmission_6hours/unique_visits_without_procedures)*100,2)
unplanned_24hrs = round((unplanned_icu_readmission_24hours/unique_visits_without_procedures)*100,2)
unplanned_48hrs = round((unplanned_icu_readmission_48hours/unique_visits_without_procedures)*100,2)
unplanned_72hrs = round((unplanned_icu_readmission_72hours/unique_visits_without_procedures)*100,2)

# Print all readmission rate
print(f"All ICU readmission ({all_icu_readmission}/{total_unique_visits}): {all_icu_read}%")
print(f"All ICU readmission <6hr ({all_icu_readmission_6hours}/{total_unique_visits}): {all_6hrs}%")
print(f"All ICU readmission <24hr ({all_icu_readmission_24hours}/{total_unique_visits}): {all_24hrs}%")
print(f"All ICU readmission <48hr ({all_icu_readmission_48hours}/{total_unique_visits}): {all_48hrs}%")
print(f"All ICU readmission <72hr ({all_icu_readmission_72hours}/{total_unique_visits}): {all_72hrs}%")

# Print unplanned readmission rate
print(f"Unplanned ICU readmission ({unplanned_icu_readmission}/{unique_visits_without_procedures}): {unplanned_icu_read}%")
print(f"Unplanned ICU readmission <6hr ({unplanned_icu_readmission_6hours}/{unique_visits_without_procedures}): {unplanned_6hrs}%")
print(f"Unplanned ICU readmission <24hr ({unplanned_icu_readmission_24hours}/{unique_visits_without_procedures}): {unplanned_24hrs}%")
print(f"Unplanned ICU readmission <48hr ({unplanned_icu_readmission_48hours}/{unique_visits_without_procedures}): {unplanned_48hrs}%")
print(f"Unplanned ICU readmission <72hr ({unplanned_icu_readmission_72hours}/{unique_visits_without_procedures}): {unplanned_72hrs}%")

In [ ]:
data = {
    'Metric': [
        'All ICU Readmission', 
        'All ICU Readmission <6hr', 
        'All ICU Readmission <24hr', 
        'All ICU Readmission <48hr', 
        'All ICU Readmission <72hr', 
        'Unplanned ICU Readmission', 
        'Unplanned ICU Readmission <6hr', 
        'Unplanned ICU Readmission <24hr', 
        'Unplanned ICU Readmission <48hr', 
        'Unplanned ICU Readmission <72hr'
    ],
    'Numerator/Denominator': [
        f"{all_icu_readmission}/{total_unique_visits}", 
        f"{all_icu_readmission_6hours}/{total_unique_visits}", 
        f"{all_icu_readmission_24hours}/{total_unique_visits}", 
        f"{all_icu_readmission_48hours}/{total_unique_visits}", 
        f"{all_icu_readmission_72hours}/{total_unique_visits}", 
        f"{unplanned_icu_readmission}/{unique_visits_without_procedures}", 
        f"{unplanned_icu_readmission_6hours}/{unique_visits_without_procedures}", 
        f"{unplanned_icu_readmission_24hours}/{unique_visits_without_procedures}", 
        f"{unplanned_icu_readmission_48hours}/{unique_visits_without_procedures}", 
        f"{unplanned_icu_readmission_72hours}/{unique_visits_without_procedures}"
    ],
    'Percentage (%)': [
        all_icu_read, 
        all_6hrs, 
        all_24hrs, 
        all_48hrs, 
        all_72hrs, 
        unplanned_icu_read, 
        unplanned_6hrs, 
        unplanned_24hrs, 
        unplanned_48hrs, 
        unplanned_72hrs
    ]
}
# Create the DataFrame
final_df = pd.DataFrame(data)
final_df.to_csv(f'{output_path}/ICU_readmission_rates_{site_name}_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv')
final_df

# Generate Sankey Plot

In [ ]:
dropped_deaths_df2 = dropped_deaths_df.rename(columns={
    'in_dttm': 'earliest_location_start',
    'out_dttm': 'latest_location_end'
})

dropped_df_icu_discharge2 = dropped_df_icu_discharge.rename(columns={
    'in_dttm': 'earliest_location_start',
    'out_dttm': 'latest_location_end'
})

print(dropped_deaths_df2["hospitalization_id"].nunique())
print(dropped_df_icu_discharge2["hospitalization_id"].nunique())

In [ ]:
icu_df7_death = pd.concat([dropped_df_icu_discharge2[["patient_id","hospitalization_id","earliest_location_start",
                                                    "latest_location_end","location_category","procedural"]], 
                           dropped_deaths_df2[["patient_id","hospitalization_id","earliest_location_start",
                                                    "latest_location_end","location_category","procedural"]]], axis=0)
icu_df7_death = pd.concat([icu_df7_death, icu_df7[["patient_id","hospitalization_id","earliest_location_start",
                                                    "latest_location_end","location_category","procedural"]]], axis=0)

icu_df7_death = pd.merge(icu_df7_death,patient[["patient_id","death_dttm"]],on="patient_id",how="left")
icu_df7_death = pd.merge(icu_df7_death,hosp[["hospitalization_id","discharge_category"]],on="hospitalization_id",how="left")


icu_df7_death.head()
print("Number of rows in df:",len(icu_df7_death))
print("Number of unique patient_id:", icu_df7_death["patient_id"].nunique())
print("Number of unique hospitalization_id:", icu_df7_death["hospitalization_id"].nunique())

In [ ]:
def mark_deaths(df):
    # Ensure the columns are in datetime format
    df['death_dttm'] = pd.to_datetime(df['death_dttm'], errors='coerce').dt.tz_localize(None)
    df['earliest_location_start'] = pd.to_datetime(df['earliest_location_start'], errors='coerce').dt.tz_localize(None)
    df['latest_location_end'] = pd.to_datetime(df['latest_location_end'], errors='coerce').dt.tz_localize(None)

    # Create the location_category_death column as a copy of location_category
    df['location_category_death'] = df['location_category']

    # Sort the dataframe by hospitalization_id and earliest_location_start to ensure order
    df = df.sort_values(by=['hospitalization_id', 'earliest_location_start'])

    # Vectorized condition to check if death_dttm is between earliest_location_start and latest_location_end
    death_condition = (
        df['death_dttm'].notna() & 
        (df['earliest_location_start'] <= df['death_dttm']) & 
        (df['death_dttm'] <= df['latest_location_end'])
    )

    # Get unique hospitalization_ids where death occurs during the stay
    death_hosp_ids = df.loc[death_condition, 'hospitalization_id'].unique()

    # Get the last row of each hospitalization where death occurs
    death_rows = df[df['hospitalization_id'].isin(death_hosp_ids)].groupby('hospitalization_id').last().reset_index()

    # Modify those rows to represent the 'Death' event
    death_rows['location_category_death'] = 'Died'
    death_rows['earliest_location_start'] = death_rows['latest_location_end'] + pd.Timedelta(seconds=1)
    death_rows['latest_location_end'] = death_rows['earliest_location_start']  # Ensure 'Death' row has valid time window

    # Concatenate the new 'Death' rows to the original dataframe
    df = pd.concat([df, death_rows], ignore_index=True)

    # Sort the dataframe again to ensure 'Death' rows come after the original locations
    df = df.sort_values(by=['hospitalization_id', 'earliest_location_start']).reset_index(drop=True)

    return df
# Apply the function to the DataFrame
icu_df7_death = mark_deaths(icu_df7_death)

# Output the results
print("Number of dead patients:", icu_df7_death[icu_df7_death["location_category_death"] == "Died"]["hospitalization_id"].nunique())
print("Number of rows in df:", len(icu_df7_death))
print("Number of unique hospitalization_id:", icu_df7_death["hospitalization_id"].nunique())


In [ ]:
icu_df7_death[icu_df7_death["location_category_death"]=="Died"][["hospitalization_id",
                                                                  "earliest_location_start",
                                                                 "latest_location_end",
                                                                 "death_dttm",
                                                                 "location_category",
                                                                 "location_category_death"]].head()

## This code chunk takes longer to run: 

In [ ]:
def collapse_segments(df):
    # Ensure data is sorted by 'earliest_location_start'
    df = df.sort_values('earliest_location_start').reset_index(drop=True)

    # Create a new column to track if the location was ICU in the original location_category
    df['is_icu_in_location_category'] = (df['location_category'] == 'ICU')

    # Detect changes in location category, both in the original and in the death-adjusted version
    df['loc_cat_change'] = (df['location_category_death'] != df['location_category_death'].shift(1)).astype(int)

    # Assign a segment_id that increments when location_category_death changes
    df['segment_id'] = df['loc_cat_change'].cumsum()

    # Group by 'segment_id' and 'location_category_death' to merge consecutive segments
    grouped = df.groupby(['segment_id', 'location_category_death']).agg({
        'earliest_location_start': 'min',
        'latest_location_end': 'max',
        'is_icu_in_location_category': 'max'  # Capture whether any part of the segment was ICU
    }).reset_index()

    # Add 'hospitalization_id' to the grouped DataFrame
    grouped['hospitalization_id'] = df['hospitalization_id'].iloc[0]

    # Rearrange columns
    grouped = grouped[['hospitalization_id', 'segment_id', 'earliest_location_start', 'latest_location_end', 'location_category_death', 'is_icu_in_location_category']]
    
    # Initialize 'segment_rank' to None
    grouped['segment_rank'] = None

    # Now we can still find ICU segments based on the original location_category, but respect Death as a location in location_category_death
    icu_segments = grouped[grouped['is_icu_in_location_category'] == 1]

    if icu_segments.empty:
        print(f"No ICU segments for hospitalization_id: {df['hospitalization_id'].iloc[0]}")
        return pd.DataFrame()  # Return empty DataFrame to exclude this hospitalization

    # Get the index of the first ICU segment
    first_icu_index = icu_segments.index[0]

    # Create 'segment_rank' starting from the first ICU segment
    grouped.loc[first_icu_index:, 'segment_rank'] = range(1, len(grouped.loc[first_icu_index:]) + 1)

    # Fill in the 'segment_rank' for non-ICU segments after the first ICU
    grouped['segment_rank'] = grouped['segment_rank'].ffill()

    # Compute the maximum segment rank for this hospitalization
    max_segment_rank = grouped['segment_rank'].dropna().max()

    # Add 'max_segment_rank' column to the DataFrame
    grouped['max_segment_rank'] = max_segment_rank

    # Drop the auxiliary column
    grouped = grouped.drop(columns=['is_icu_in_location_category'])

    return grouped

# Apply the function to the DataFrame
collapsed_adt = icu_df7_death[icu_df7_death["procedural"] == False][['hospitalization_id', 'earliest_location_start', 'latest_location_end', 'location_category', 'location_category_death']] \
    .groupby('hospitalization_id', group_keys=False).apply(collapse_segments).reset_index(drop=True)

print("Number of unique hospitalization_id:", collapsed_adt["hospitalization_id"].nunique())
print("Number of rows in df:", len(collapsed_adt))

In [ ]:
def drop_intermediate_er2(df):
    # Sort the DataFrame to ensure correct order of rows within each hospitalization_id
    df = df.sort_values(by=['hospitalization_id', 'earliest_location_start', 'latest_location_end'])

    # Create a boolean mask to detect if 'ER' appears after the first row in each hospitalization
    df['is_first_row'] = df.groupby('hospitalization_id').cumcount() == 0

    # Group by 'hospitalization_id' and check if any 'ER' appears after the first row
    er_after_first_row = df.groupby('hospitalization_id').apply(
        lambda group: (group.loc[~group['is_first_row'], 'location_category_death'] == 'ER').any()
    )

    # Find hospitalization_ids where 'ER' appears after the first row
    visits_to_drop = er_after_first_row[er_after_first_row].index

    # Drop rows from the original DataFrame where hospitalization_id is in visits_to_drop
    df_cleaned = df[~df['hospitalization_id'].isin(visits_to_drop)]
    dropped_df = df[df['hospitalization_id'].isin(visits_to_drop)]

    # Drop the 'is_first_row' helper column
    df_cleaned = df_cleaned.drop(columns=['is_first_row'])

    return df_cleaned, dropped_df

# Apply the optimized function to the DataFrame
collapsed_adt2, dropped_df = drop_intermediate_er2(collapsed_adt)

# Output the results
print("Number of unique hospitalization_id:", collapsed_adt2["hospitalization_id"].nunique())
print("Number of rows in df:", len(collapsed_adt2))


In [ ]:
sankey_df = collapsed_adt2.reset_index().loc[:,['hospitalization_id','segment_rank','location_category_death']]
sankey_df2 = sankey_df.loc[sankey_df.segment_rank.notna()] 
sankey_df3 = sankey_df2.pivot(
    index=['hospitalization_id'],
    columns='segment_rank',
    values='location_category_death'
).reset_index().fillna('Discharged')
sankey_df4 = sankey_df3.iloc[:, :8] #only take first 7 locations, otherwise gets too long

In [ ]:
def propagate_death(df):
    # List of columns that represent the different segments
    segment_cols = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]

    # Define a function to propagate 'Died' across the columns
    def propagate(row):
        death_found = False
        for col in segment_cols:
            if row[col] == 'Died':
                death_found = True
            if death_found:
                row[col] = 'Died'
        return row

    # Apply the function to each row
    df[segment_cols] = df[segment_cols].apply(propagate, axis=1)

    return df

# Apply the function to propagate 'Death' to subsequent columns
sankey_df4 = propagate_death(sankey_df4)

In [ ]:
colors = {
    "ICU": 'lightcoral',
    "Ward": 'skyblue',
    'Procedural':'thistle',
    "Discharged": 'lightgrey',
    "Died": 'grey',
    "ER":'red'
}
fig, ax = plt.subplots(figsize=(14, 6), constrained_layout=True)
diag = Sankey(
    sankey_df4.iloc[:, 1:], 
    ax=ax, 
    order=["ICU", "Ward", "Procedural", "Discharged","ER","Died"],
    block_width=0.2,
    colors=colors,
    alpha=0.5
)
diag.draw()
ax.set_title("", size=16)
ax.set_xticks(
    [diag.block_width / 2 + diag.flow_width * x + diag.block_width * x for x in range(sankey_df4.shape[1] - 1)]
)
ax.set_xticklabels(sankey_df4.columns[1:].astype(int))
ax.set_xlabel("Location number", size=14)
ax.get_xaxis().set_visible(True)
ax.tick_params(axis="x", pad=5, labelsize=16)

# Export the figure as a high-resolution image
fig.savefig(f'{output_path}/sankey_diagram_{site_name}_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.png', dpi=300, bbox_inches='tight')

# Generate Table One

In [ ]:
unplanned_icu_final = icu_final[icu_final["procedural"]==False]
#admission location
admission_location = icu_df7[["hospitalization_id", "location_category"]].groupby('hospitalization_id', as_index=False).first()
unplanned_icu_final = pd.merge(unplanned_icu_final,admission_location,on="hospitalization_id",how="left")

#length of ICU stay 
icu_sorted = icu_df7.sort_values(by=['hospitalization_id', 'earliest_location_start'])
icu_first_icu = icu_sorted[icu_sorted['location_category'] == 'ICU'].groupby('hospitalization_id', as_index=False).first()
length_ICU = icu_first_icu[['hospitalization_id', 'location_hours']]
unplanned_icu_final = pd.merge(unplanned_icu_final,length_ICU,on="hospitalization_id",how="left")
unplanned_icu_final['location_days'] = unplanned_icu_final['location_hours'] / 24

#in-hospital mortality
dead_patients = collapsed_adt2[collapsed_adt2["location_category_death"]=="Died"][["hospitalization_id","location_category_death"]]
unplanned_icu_final = pd.merge(unplanned_icu_final,dead_patients,on="hospitalization_id",how="left")
unplanned_icu_final['location_category_death'] = unplanned_icu_final['location_category_death'].fillna('Alive')

#age at admission
hosp = hosp.drop_duplicates(subset=["hospitalization_id"])
age = hosp[["hospitalization_id","age_at_admission"]]                           
unplanned_icu_final = pd.merge(unplanned_icu_final,age,on="hospitalization_id",how="left")

#demographics 
demog = patient[["patient_id","race_category","ethnicity_category","sex_category"]]
unplanned_icu_final = pd.merge(unplanned_icu_final,demog,on="patient_id",how="left")
unplanned_icu_final = unplanned_icu_final.drop_duplicates(subset=["hospitalization_id"])

#readmitted
unplanned_icu_final['readmission'] = unplanned_icu_final['ICU_readmission_hour'].apply(
    lambda x: 'Readmitted' if pd.notna(x) and x > 0 else 'Not Readmitted'
)

print(unplanned_icu_final["hospitalization_id"].nunique())
unplanned_icu_final.head()

In [ ]:
table_one_dat = unplanned_icu_final[["age_at_admission",
                                     "sex_category",
                                     "race_category",
                                    "ethnicity_category",
                                    "location_category",
                                     "ICU_readmission_hour",
                                    "location_days",
                                    "location_category_death",
                                    "readmission"]]

# Rename columns
table_one_dat = table_one_dat.rename(columns={'age_at_admission': 'Age', 
                                              'sex_category': 'Sex', 
                                              'race_category': 'Race',
                                             'ethnicity_category': 'Ethnicity',
                                             'location_category': 'Initial hospital location',
                                              'ICU_readmission_hour':'ICU readmission, hours',
                                              "location_days":'ICU length of stay, days',
                                              "location_category_death":'In-hospital Mortality'
                                             })

In [ ]:
# Define the columns that include both continuous and categorical variables
columns = ["Age", "ICU readmission, hours", "ICU length of stay, days", "Sex", "Race", "Ethnicity", "Initial hospital location", "In-hospital Mortality"]

# Define which columns are categorical
categorical = ["Sex", "Race", "Ethnicity", "Initial hospital location", "In-hospital Mortality"]

# Define which continuous variables are not normally distributed
nonnormal = ["Age","ICU readmission, hours","ICU length of stay, days"]

#groupby
groupby = 'readmission'

# Generate the TableOne object
table1 = TableOne(data=table_one_dat, 
                  columns=columns, 
                  categorical=categorical, 
                  nonnormal=nonnormal, 
                  groupby=groupby)

# Display the table

table1.to_csv(f'{output_path}/table1_{site_name}_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.csv')
table1